In [ ]:
from confluent_kafka import Producer
import requests
import json
import time
import pandas as pd

url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    'vs_currency': 'usd',
    'order': 'market_cap_desc',
    'per_page': 100,
    'page': 1,
    'sparkline': False
}
# Kafka producer setup for streaming data
producer = KafkaProducer(bootstrap_servers='localhost:9092',
                         value_serializer=lambda v: json.dumps(v).encode('utf-8'))

# Initialize an empty DataFrame to store data
all_data = pd.DataFrame()

while True:
    # Fetch data from the API
    response = requests.get(url, params=params)
    data = response.json()
    
    # Convert the data to a DataFrame and append to existing data
    df = pd.DataFrame(data)
    all_data = pd.concat([all_data, df], ignore_index=True)
    
    # Save the data to a CSV file
    all_data.to_csv('kafka_crypto_data.csv', index=False)
    
    # Send the data to the Kafka topic
    producer.send('crypto-topic', data)
    
    # Wait for one minute before fetching new data
    time.sleep(60)